# 乱数を使って保持器を作る

In [18]:
import numpy as np
import pandas as pd

In [19]:
np.random.seed(0)
params = np.random.randn(20, 10) / 3
params[params<-1] = -1
params[params>1] = 1
params[:2]

array([[ 0.58801745,  0.13338574,  0.32624599,  0.7469644 ,  0.62251933,
        -0.32575929,  0.31669614, -0.0504524 , -0.03440628,  0.13686617],
       [ 0.04801452,  0.48475784,  0.25367924,  0.04055834,  0.14795441,
         0.11122478,  0.49802636, -0.06838609,  0.1043559 , -0.28469858]])

In [20]:
params.max(), params.min()

(0.794381591621314, -0.8509966052780262)

In [21]:
r0 = lambda dr: 2.35e-3 + dr
r1 = lambda dr: 2.85e-3 + dr

r0(0.25e-3), r1(-0.25e-3)

(0.0026, 0.0026)

In [22]:
r0s = r0(params[:, 0] * 0.2e-3)
r1s = r1(params[:, 1] * 0.2e-3)
drs = r1s - r0s

drs.min() * 1e3, drs.max() * 1e3, r0s.min() * 1e3, r1s.max() * 1e3


(0.2846060161200868, 0.7137738940849623, 2.179800678944395, 2.9786354702544657)

In [23]:
h0 = lambda dh: 0.93e-3 + dh
h1 = lambda dh: 2.10e-3 + dh

h0s = h0(params[:, 2] * 0.2e-3)
h1s = h1(params[:, 3] * 0.2e-3)

h0s.max() * 1e3, h0s.min() * 1e3, h1s.max() * 1e3, h1s.min() * 1e3

(1.00596004563622, 0.8162486539583326, 2.2493928799467637, 1.9517731231850382)

In [24]:
bevel = lambda r: 0.1 * drs * r

bevels = bevel(params[:, 4]+2)

(bevels / drs).max(), (bevels / drs).min()

(0.27943815916213144, 0.15915214937232774)

In [25]:
pocket_r = lambda dr: 0.825e-3 + dr

pocket_rs = pocket_r(params[:, 5] * 1e-6)
pocket_rs.min() * 1e3, pocket_rs.max() * 1e3

(0.8244659807814644, 0.8256478737285498)

In [26]:
neck_x = lambda dx: 1.70e-3 + dx

neck_xs = neck_x(params[:, 6] * 20e-6)
neck_xs.min() * 1e3, neck_xs.max() * 1e3

(1.6891320110202264, 1.7099605271543838)

In [27]:
neck_r = lambda dr: 1.20e-3 + dr

neck_rs = neck_r(params[:, 7] * 20e-6)
neck_rs.min() * 1e3, neck_rs.max() * 1e3


(1.1897015260214825, 1.2123284248565287)

In [28]:
neck_h = lambda dh: 2.45e-3 + dh

neck_hs = neck_h(params[:, 8] * 100e-6)
neck_hs.min() * 1e3, neck_hs.max() * 1e3


(2.3962034050814016, 2.522107864976023)

In [29]:
neck_dh = lambda r: (neck_hs - h1s) * r * 0.1

neck_dhs = neck_dh(params[:, 9]+2)
neck_dhs.min() * 1e3, neck_dhs.max() * 1e3


(0.03205210368709177, 0.12227076184407988)

In [30]:
df = pd.DataFrame({
    'r0': r0s,
    'r1': r1s,
    'h0': h0s,
    'h1': h1s,
    'bevel': bevels,
    'pocket_r': pocket_rs,
    'neck_x': neck_xs,
    'neck_r': neck_rs,
    'neck_h': neck_hs,
    'neck_dh': neck_dhs
})

df.to_csv('../data/random/random_params.csv', index=False)


In [38]:
# cargo run --release "data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace.vtu" "2.345e-3,2.850e-3,0.93e-3,2.10e-3,0.10e-3,0.825e-3,1.70e-3,1.20e-3,2.45e-3,0.152e-3" "100"

command_args0 = lambda i: f"\"data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace{i}.vtu\" "
command_args0(0)


'"data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace0.vtu" '

In [41]:
command_args1 = lambda i: f'\"{r0s[i]},{r1s[i]},{h0s[i]},{h1s[i]},{bevels[i]},{pocket_rs[i]},{neck_xs[i]},{neck_rs[i]},{neck_hs[i]},{neck_dhs[i]}\" \"1000\"'
command_args1(10)

# '../data/Tetra_linspace10.vtu 0.0024755433798037503 0.002760149395923837 0.0008453010001009512 0.002164626447210534 4.579193726579163e-05 0.0008256478737285498 0.0016972425401282683 0.0011950169679237283 0.002514098067549346 0.0001742818416754026 1000'

'"0.0024755433798037503,0.002760149395923837,0.0008453010001009512,0.002164626447210534,4.579193726579163e-05,0.0008256478737285498,0.0016972425401282683,0.0011950169679237283,0.002514098067549346,8.71409208377013e-05" "1000"'

In [42]:
import subprocess

command = "cd ../; cargo run --release "
subprocess.run(command + command_args0(10) + command_args1(10), shell=True)


   Compiling mophing v0.1.0 (/Users/gyakusu/Morphing)
    Finished release [optimized] target(s) in 1.19s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace10.vtu 0.0024755433798037503,0.002760149395923837,0.0008453010001009512,0.002164626447210534,4.579193726579163e-05,0.0008256478737285498,0.0016972425401282683,0.0011950169679237283,0.002514098067549346,8.71409208377013e-05 1000`


[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace10.vtu"], ["0.0024755433798037503", "0.002760149395923837", "0.0008453010001009512", "0.002164626447210534", "4.579193726579163e-05", "0.0008256478737285498", "0.0016972425401282683", "0.0011950169679237283", "0.002514098067549346", "8.71409208377013e-05"], ["1000"]]
The new file has been created: data/Tetra_linspace10.vtu


CompletedProcess(args='cd ../; cargo run --release "data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace10.vtu" "0.0024755433798037503,0.002760149395923837,0.0008453010001009512,0.002164626447210534,4.579193726579163e-05,0.0008256478737285498,0.0016972425401282683,0.0011950169679237283,0.002514098067549346,8.71409208377013e-05" "1000"', returncode=0)

In [43]:
for i in range(len(params)):
    subprocess.run(command + command_args0(i) + command_args1(i), shell=True)
    print(f"Done {i}")


    Finished release [optimized] target(s) in 0.13s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace0.vtu 0.0024676034897311776,0.0028766771472244815,0.0009952491989403827,0.0022493928799467637,0.00010728035741904383,0.0008246742407067079,0.0017063339227835039,0.001198990951944682,0.002446559371606881,4.213184053562646e-05 1000`


[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace0.vtu"], ["0.0024676034897311776", "0.0028766771472244815", "0.0009952491989403827", "0.0022493928799467637", "0.00010728035741904383", "0.0008246742407067079", "0.0017063339227835039", "0.001198990951944682", "0.002446559371606881", "4.213184053562646e-05"], ["1000"]]
The new file has been created: data/Tetra_linspace0.vtu
Done 0
[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace1.vtu"], ["0.0023596029047440585", "0.002946951567130865", "0.0009807358483431329", "0.0021081116677661885", "0.00012615981501188495", "0.0008251112247757914", "0.001709960527154384", "0.0011986322782415611", "0.00246043559005503", "6.043417242840259e-05"], ["1000"]]


    Finished release [optimized] target(s) in 0.04s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace1.vtu 0.0023596029047440585,0.002946951567130865,0.0009807358483431329,0.0021081116677661885,0.00012615981501188495,0.0008251112247757914,0.001709960527154384,0.0011986322782415611,0.00246043559005503,6.043417242840259e-05 1000`


The new file has been created: data/Tetra_linspace1.vtu
Done 1
[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace2.vtu"], ["0.002179800678944395", "0.0028935745730293574", "0.000987629079923967", "0.0020505223319729035", "0.0001967578320363586", "0.0008245152114418004", "0.0017003050567820095", "0.0011987521076664944", "0.0025010926404786154", "0.00011218237617646054"], ["1000"]]


    Finished release [optimized] target(s) in 0.03s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace2.vtu 0.002179800678944395,0.0028935745730293574,0.000987629079923967,0.0020505223319729035,0.0001967578320363586,0.0008245152114418004,0.0017003050567820095,0.0011987521076664944,0.0025010926404786154,0.00011218237617646054 1000`


...

[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace17.vtu"], ["0.0024113905882520545", "0.0028712485101962015", "0.0009871220407935129", "0.00205659829377999", "7.611809225520898e-05", "0.0008252271981727605", "0.0016946439355721742", "0.0011954030014816653", "0.0024348155832160886", "7.58638218921557e-05"], ["1000"]]
The new file has been created: data/Tetra_linspace17.vtu
Done 17
[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace18.vtu"], ["0.0023264004059164343", "0.0027583365804387987", "0.0008870921064778074", "0.001951773123185038", "9.538923760939789e-05", "0.0008244659807814644", "0.0016926374444038103", "0.0012003477671950731", "0.0024253479001202895", "0.00011907271514796971"], ["1000"]]


    Finished release [optimized] target(s) in 0.04s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace18.vtu 0.0023264004059164343,0.0027583365804387987,0.0008870921064778074,0.001951773123185038,9.538923760939789e-05,0.0008244659807814644,0.0016926374444038103,0.0012003477671950731,0.0024253479001202895,0.00011907271514796971 1000`


The new file has been created: data/Tetra_linspace18.vtu
Done 18
[["data/Tetra.vtu", "data/face_and_edge_index.xml", "data/Tetra_linspace19.vtu"], ["0.00226380953935177", "0.0028678033912899457", "0.0009273811454515004", "0.002022127100150587", "0.00013133396658176577", "0.0008249428178895926", "0.0017051452703414243", "0.0012054900276930913", "0.002522107864976023", "0.00012227076184407988"], ["1000"]]


    Finished release [optimized] target(s) in 0.03s
     Running `target/release/mophing data/Tetra.vtu,data/face_and_edge_index.xml,data/Tetra_linspace19.vtu 0.00226380953935177,0.0028678033912899457,0.0009273811454515004,0.002022127100150587,0.00013133396658176577,0.0008249428178895926,0.0017051452703414243,0.0012054900276930913,0.002522107864976023,0.00012227076184407988 1000`


The new file has been created: data/Tetra_linspace19.vtu
Done 19
